Using **Langchain**

In [1]:
pip install pandas scikit-learn


In [2]:
import pandas as pd

# Load the Excel file
xlsx_path = '/content/GPT2_finetune_new CSV.xlsx'  # Adjust this to the path of your Excel file
df = pd.read_excel(xlsx_path)

# Convert the DataFrame to a single string with each row's values concatenated
# You can change the separator if needed
rows_as_strings = df.apply(lambda row: ' '.join(row.astype(str)), axis=1)

# Concatenate all rows into a single text, with each row separated by a newline character
all_text = '\n'.join(rows_as_strings)

# Write the concatenated text to a file
txt_path = '/content/new.txt'  # Adjust the path and filename as needed
with open(txt_path, 'w', encoding='utf-8') as text_file:
    text_file.write(all_text)

txt_path  # Return the path of the created text file for reference


'/content/new.txt'

In [3]:
# Assuming we have the text from the Excel file converted and saved in "converted_file.txt",
# let's load the text file and apply a train-test split.

txt_path = '/content/new.txt'  # Path to the converted text file

# Load the entire text file
with open(txt_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Since each line represents a row from the original Excel file,
# we can directly split these lines into train and test sets.
from sklearn.model_selection import train_test_split

train_lines, test_lines = train_test_split(lines, test_size=0.2, random_state=42)

# Now, let's write the train and test lines to separate text files.
train_txt_path = '/content/train_txt'
test_txt_path = '/content/test_txt'

with open(train_txt_path, 'w', encoding='utf-8') as train_file:
    train_file.writelines(train_lines)

with open(test_txt_path, 'w', encoding='utf-8') as test_file:
    test_file.writelines(test_lines)

train_txt_path, test_txt_path  # Returning paths to the train and test text files for reference


('/content/train_txt', '/content/test_txt')

In [4]:

pip install torch transformers


In [5]:
!pip install accelerate -U

In [6]:
!pip install transformers[torch]

In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Prepare your dataset
train_path = r'/content/train_txt'
test_path = r'/content/test_txt'

train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=128)

test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=test_path,
    block_size=128)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# Load pre-trained model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Start training
trainer.train()

# Save the fine-tuned model
trainer.save_model("./gpt2_finetuned")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


In [9]:
import pandas as pd

# Load the Excel file
xlsx_file = "/content/GPT2_finetune_new CSV.xlsx"
df = pd.read_excel(xlsx_file)

# Save the DataFrame to a CSV file
csv_file = "output.csv"
df.to_csv(csv_file, index=False)


In [24]:
!pip install -q langchain openai chromadb

In [25]:

from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os

In [26]:

os.environ["OPENAI_API_KEY"] = "sk-proj-SgyZeKBsuyAW4YqeOJ56T3BlbkFJ5AfpoBp6Fa78L8reIfjC"

In [27]:

# Load the documents
loader = CSVLoader(file_path='/content/output.csv')

In [28]:
pip install tiktoken

In [29]:

# Create an index using the loaded documents
index_creator = VectorstoreIndexCreator()
docsearch = index_creator.from_loaders([loader])

In [30]:
# Create a question-answering chain using the index
chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")



/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [31]:

# Pass a query to the chain
query = "Do you have a column called Name?"
response = chain({"question": query})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [32]:

print(response['result'])

 Yes, all of the pieces of context provided include a name for the individual profiled on the Johns Hopkins Medicine website.


In [35]:
# Pass a query to the chain
query = "Will you provide me all the information about Dr.Fanny Meylin Caballeros Lam ?"
response = chain({"question": query})

In [36]:
print(response['result'])

 Yes, Dr. Fanny Meylin Caballeros Lam is a specialist in the Radiology Service at the CUN (Clinica Universidad de Navarra) and her department link is https://www.cun.es/en/our-professionals/medical-services/radiology. Additionally, you can find her professional profile and photo at the following page link: https://www.cun.es/nuestros-profesionales/profesionales/fanny-meylin-caballero-lam and her photo link: https://www.cun.es/.imaging/mte/newstyle/profesional_carrusel-80x80/dam/cun/imagen/profesionales/radiologia/dra-fanny-meylin-caballeros-lam/jcr:content/dra-fanny-caballeros.webp.


In [37]:
# Pass a query to the chain
query = "Provide me names for doctor who are specialist in Radiology Service?"
response = chain({"question": query})

In [38]:
print(response['result'])

 Dr. Fanny Meylin Caballeros Lam, Dr. David Cano Rafart, Dr. Marta Calvo Imirizaldu.


**Fine tune gpt2 model**

In [39]:

pip install langchain transformers


In [40]:
pip install openpyxl

In [41]:
import pandas as pd

# Load your dataset; assuming it's an Excel file based on the extension
df = pd.read_excel('/content/GPT2_finetune_new CSV.xlsx')

# Concatenate the columns into a single prompt string per row
df['prompt'] = df.apply(lambda x: f"Page Link: {x['Page link']} | Link: {x['link']} | Name: {x['Name']} | Category: {x['category']} | Department: {x['department']} | Department Link: {x['department link']} | Information: {x['Information']}", axis=1)

# Save the prompts to a text file, one prompt per line
df['prompt'].to_csv('training_data.txt', index=False, header=False)


In [42]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load pre-trained model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Prepare dataset and data collator
train_dataset = TextDataset(
  tokenizer=tokenizer,
  file_path='training_data.txt',
  block_size=128
)

data_collator = DataCollatorForLanguageModeling(
  tokenizer=tokenizer, mlm=False
)

# Training arguments
training_args = TrainingArguments(
  output_dir='./gpt2_finetuned',
  overwrite_output_dir=True,
  num_train_epochs=4,
  per_device_train_batch_size=4,
)

# Initialize Trainer
trainer = Trainer(
  model=model,
  args=training_args,
  data_collator=data_collator,
  train_dataset=train_dataset,
)

# Start training
trainer.train()

# Save the fine-tuned model
trainer.save_model("./gpt2_finetuned")


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


In [ ]:
# pip install transformers[torch]

In [ ]:
# pip install accelerate -U

In [43]:
from sklearn.model_selection import train_test_split

# Assuming 'data' contains your preprocessed text data loaded from 'training_data.txt'
with open('training_data.txt', 'r', encoding='utf-8') as file:
    data = file.readlines()

# Split data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)  # Adjust test_size as needed

# Save the split data back to files for training and validation
with open('train_data.txt', 'w', encoding='utf-8') as file:
    file.writelines(train_data)

with open('val_data.txt', 'w', encoding='utf-8') as file:
    file.writelines(val_data)


In [44]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load pre-trained model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Prepare dataset and data collator
train_dataset = TextDataset(tokenizer=tokenizer, file_path='train_data.txt', block_size=128)
val_dataset = TextDataset(tokenizer=tokenizer, file_path='val_data.txt', block_size=128)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir='./gpt2_finetuned2',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    evaluation_strategy="epoch",  # Perform evaluation at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Specify the validation dataset here
)

# Start training
trainer.train()

# Save the fine-tuned model
trainer.save_model("./gpt2_finetuned2")


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.331795
2,No log,1.071182
3,No log,1.017012


In [45]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load pre-trained model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Prepare dataset and data collator
train_dataset = TextDataset(tokenizer=tokenizer, file_path='train_data.txt', block_size=128)
val_dataset = TextDataset(tokenizer=tokenizer, file_path='val_data.txt', block_size=128)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir='./gpt2_finetuned3',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    evaluation_strategy="epoch",  # Perform evaluation at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=50,  # Log training loss every 50 steps
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Specify the validation dataset here
)

# Start training
trainer.train()

# Save the fine-tuned model
trainer.save_model("./gpt2_finetuned3")


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.331795
2,2.107000,1.071182
3,2.107000,1.017012


In [50]:
from transformers import pipeline

# Load your fine-tuned model
model_path = "./gpt2_finetuned2"

# Initialize the generator to use CPU
generator = pipeline('text-generation', model=model_path, tokenizer='gpt2')  # Defaults to CPU

# Test prompts
prompts = [
    "how many cancer specialist are there?",
    "How to I get there website link?"
]

# Generate answers
for prompt in prompts:
    outputs = generator(prompt, max_length=50, num_return_sequences=1)
    for i, output in enumerate(outputs):
        print(f"Prompt: {prompt}\nGenerated: {output['generated_text']}\n")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: how many cancer specialist are there?
Generated: how many cancer specialist are there?
Cheryla Lai: I am an oncologist, and I treat patients at the National Cancer Institute of China Cancer Center, at Stanford Hospital.
Oncologist: Cun.org |

Prompt: How to I get there website link?
Generated: How to I get there website link?
Websites for information about the Department's operations in Iraq and Syria: https://www.cun.es/en/our-professionals/doctors-medical-staff/prof-



In [49]:
from transformers import pipeline

# Load your fine-tuned model
model_path = "./gpt2_finetuned"

# Initialize the generator to use CPU
generator = pipeline('text-generation', model=model_path, tokenizer='gpt2')  # Defaults to CPU

# Test prompts
prompts = [
    "where I can find Anesthesia and Intensive Care Department ? "
]

# Generate answers
for prompt in prompts:
    outputs = generator(prompt, max_length=50, num_return_sequences=1)
    for i, output in enumerate(outputs):
        print(f"Prompt: {prompt}\nGenerated: {output['generated_text']}\n")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: where I can find Anesthesia and Intensive Care Department ? 
Generated: where I can find Anesthesia and Intensive Care Department ? ................................ | Name: Dr.
Barbara Anesto | Category: Specialist | Department: Anesthesia and Intensive Care Department | Department Link: https://www.hopkins



In [53]:
from transformers import pipeline

# Load your fine-tuned model
model_path = "./gpt2_finetuned"

# Initialize the generator to use CPU
generator = pipeline('text-generation', model=model_path, tokenizer='gpt2')  # Defaults to CPU

# Test prompts
prompts = [
    "where I can find Dr. Elena Cacho Asenjo [SP]? "
]

# Generate answers
for prompt in prompts:
    outputs = generator(prompt, max_length=50, num_return_sequences=1)
    for i, output in enumerate(outputs):
        print(f"Prompt: {prompt}\nGenerated: {output['generated_text']}\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: where I can find Dr. Elena Cacho Asenjo [SP]? 
Generated: where I can find Dr. Elena Cacho Asenjo [SP]?  I'm a medical director on the Pediatric Oncology Department at Memorial Sloan-Kettering Cancer Center in New York City and Dr. Ana Botello



In [62]:
from transformers import pipeline

# Load your fine-tuned model
model_path = "./gpt2_finetuned3"

# Initialize the generator to use CPU
generator = pipeline('text-generation', model=model_path, tokenizer='gpt2')  # Defaults to CPU

# Test prompts
prompts = [
    "where I can find Palliative Medicine Unit give me some information ? "
]

# Generate answers
for prompt in prompts:
    outputs = generator(prompt, max_length=50, num_return_sequences=1)
    for i, output in enumerate(outputs):
        print(f"Prompt: {prompt}\nGenerated: {output['generated_text']}\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: where I can find Palliative Medicine Unit give me some information ? 
Generated: where I can find Palliative Medicine Unit give me some information ? ~~ Help is on the way.~~ [SP] | Information: Palliative Medicine Unit"
Kirsty Tappey | Category: Specialist | Department: Head



In [64]:
from transformers import pipeline

# Load your fine-tuned model
model_path = "./gpt2_finetuned3"

# Initialize the generator to use CPU
generator = pipeline('text-generation', model=model_path, tokenizer='gpt2')  # Defaults to CPU

# Test prompts
prompts = [
    "  The specialist Dr. Ricardo Calderón Pelayo [SP] works on which category  ? "
]

# Generate answers
for prompt in prompts:
    outputs = generator(prompt, max_length=50, num_return_sequences=1)
    for i, output in enumerate(outputs):
        print(f"Prompt: {prompt}\nGenerated: {output['generated_text']}\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt:   The specialist Dr. Ricardo Calderón Pelayo [SP] works on which category  ? 
Generated:   The specialist Dr. Ricardo Calderón Pelayo [SP] works on which category  ? --------------------------- Aphrodisiacal Medicine Department [SP] Status: Visiting Specialist | Department: Ophthalmology Department | Department Link:

